In [1]:
import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
dist_path = './22_12_26_Выборы_студсоветов_и_УПС_Распределение_по_участкам_База.xlsx'
districts = pd.read_excel(dist_path, sheet_name='Распределение')
voters = pd.read_excel(dist_path, sheet_name='База')
votes = pd.read_excel('./22_12_28_Выборы_студсоветов_и_УПС_Выгрузка.xlsx', skiprows=[1, 2])
votes = votes.merge(voters, on='ac')
candidates = pd.read_excel('./candidates.xlsx')
candidates = candidates.merge(districts[['Подразделение', 'код']], left_on='Подразделение', right_on='Подразделение', how='left')

In [ ]:
# проверка на адекватность
nums = ['first', 'second', 'third', 'fourth', 'fifth']
extras = [f'extra_{x}' for x in nums]
for index, row in votes.iterrows():
    # проверка на допустимость округов
    if not all([row[x]==row[y] or (pd.isna(row[x]) and pd.isna(row[y])) for x, y in zip(nums, extras)]):
        print('НЕСООТВЕТСТВИЕ ОКРУГОВ')
        print(row['№ записи'])
        print([row[x] for x in nums])
        print([row[x] for x in extras])

    # проверка на допустимость кандидатов
    allowed_codes = [x for x in row[extras] if not pd.isna(x)]
    non_null_candidates = [x for x in candidates['id'] if not (pd.isna(row[x]) or row[x] in ('', ' '))]
    non_null_candidate_codes = [x for x in candidates[candidates['id'].isin(non_null_candidates)]['код']]
    if not all([x in allowed_codes for x in non_null_candidate_codes]):
        print('ГОЛОСА ЗА НЕДОПУСТИМЫХ КАНДИДАТОВ')
        print(row['№ записи'])
        print(allowed_codes)
        print(non_null_candidates)
        print(non_null_candidate_codes)
    
    # проверка на валидность количества голосов
    code_sums = {}
    for code, candidate in zip(non_null_candidate_codes, non_null_candidates):
        if code == '27m8d':
            continue # за омбудсмена голоса не суммируются
        if code not in code_sums:
            code_sums[code] = 0
        code_sums[code] += row[candidate]
    if not all([summ == districts[districts['код'] == code]['Мандаты'].values[0] or summ==0 for code, summ in code_sums.items()]):
        print('НЕ СХОДИТСЯ СУММА ГОЛОСОВ')
        print(row['№ записи'])
        print(code_sums)


In [ ]:
# если каждый отдельный голос валиден, то можно считать результаты просто суммируя голоса по столбцам (кроме омбудсмена)
cols = [x for x in candidates['id'].tolist() if x != 'Qu35'] # убираем омбудсмена

# преобразуем все в числа, по дефолту Object
votes[cols] = votes[cols].fillna(0).apply(pd.to_numeric, errors='coerce')

In [ ]:
results = votes[cols].sum(numeric_only=True, axis=0, skipna=True).to_frame().rename({0: 'Голосов'}, axis=1)
results = results.merge(candidates, left_index=True, right_on='id', how='left')
results['Проголосовавших'] = results['id'].apply(lambda x: votes[x].value_counts().drop(0.0, errors='ignore').sum())

results.sort_values(by=['Подразделение', 'Голосов', 'Проголосовавших'], ascending=False)[
    ['Подразделение', 'Имя', 'Голосов']].to_excel('results_tables/results.xlsx', index=False)
results.to_excel('results_tables/results_full.xlsx', index=False)

In [ ]:
votes_with_mief = votes.copy()
votes_with_mief['first'].fillna('mief', inplace=True) # у миэф нет графы first, но за омбудсмена они голосуют
ombudsman_by_op = votes_with_mief.groupby(['Qu35', 'first']).count().rename({1: 'Морозов', 2: 'Варванская'}).loc[['Морозов', 'Варванская']]['ac']
ombudsman_by_op = ombudsman_by_op.unstack(level=0)
ombudsman_by_op = ombudsman_by_op.reset_index()
ombudsman_by_op = ombudsman_by_op.merge(districts[['Подразделение', 'код']], left_on='first', right_on='код', how='left')
ombudsman_by_op.index = ombudsman_by_op['Подразделение']
ombudsman_by_op = ombudsman_by_op[['Морозов', 'Варванская']]

ombudsman_by_op.to_excel('results_tables/ombudsman_by_op.xlsx')

In [ ]:
passed_color = '#ff9b21'
failed_color = '#d9d9d9'
morozov_color = '#901de8'
varvanskaia_color = '#ef553c'


with pd.ExcelWriter('results_tables/results_pretty.xlsx') as writer:
    for district, acronym in districts[['Подразделение', 'Аббревиатура']].values:
        mandates = districts[districts['Подразделение'] == district]['Мандаты'].values[0]
        district_results = results[results['Подразделение'] == district]\
                .sort_values(by=['Голосов', 'Проголосовавших'], ascending=False)[['Голосов', 'Имя']]
        total = len(district_results)

        styler = district_results.style.apply(lambda row: pd.Series('background-color: yellow', row.index) \
                                            if row['Имя'] in district_results.head(mandates)['Имя'].values \
                                            else pd.Series('', row.index), axis=1)
        styler.to_excel(writer, sheet_name=acronym, index=False)

        try:
            if district_results.iloc[mandates]['Голосов'] == district_results.iloc[mandates - 1]['Голосов']:
                print('СОВПАДЕНИЕ ГОЛОСОВ') # выводим для ручной перепроверки
                print(district)
                raw_results = results[results['Подразделение'] == district]\
                    .sort_values(by=['Голосов', 'Проголосовавших'], ascending=False)
                print(raw_results.iloc[mandates - 1:mandates + 1][['Проголосовавших', 'Голосов', 'Имя']])
        except Exception:
            pass

        try:
            ombudsman_district = ombudsman_by_op.loc[district]
        except KeyError:
            ombudsman_district = None # омбудсмена нет в кампусах, общежитиях, СОИС

        if ombudsman_district is not None:
            omb_fig = go.Figure()
            omb_fig.add_trace(
                go.Bar(
                    x=['Данила Морозов', 'Арина Варванская'],
                    y=ombudsman_district.values,
                    marker_color=[morozov_color, varvanskaia_color],
                    width=0.5,
                    text=ombudsman_district.values,
                )
            )
            omb_fig.update_layout(
                showlegend=False,
                title_text='<b>'+district.replace(' / ', '<br>')+'</b><br>Уполномоченный по правам студентов и аспирантов',
                font_family="Montserrat",
            )
            omb_fig.write_image(f'results/{district.split(" / ")[0]} УПС.png', width=800, height=400, scale=3)


        fig = go.Figure()
        fig.add_trace(
            go.Table(
                columnwidth=[35, 400],
                header=dict(
                    values=[f'<b>{x}</b>' for x in district_results.columns],
                    font=dict(size=14),
                    align="center",
                    fill_color=failed_color,
                ),
                cells=dict(
                    values=district_results.T.values,
                    align=["center", "left"],
                    fill_color=[[passed_color]*mandates + [failed_color]*(len(district_results)-mandates)]
                )
            )
        )

        base_height = 227
        row_height = 20
        long_row_count = len([x for x in district_results.T.values if len(str(x)) > 70])
        height = base_height + row_height*(total+long_row_count)

        fig.update_layout(
            height=height,
            showlegend=False,
            title_text='<b>'+district.replace(' / ', '<br>')+'</b>',
            font_family="Montserrat"
        )
        fig.write_image(f'results/{district.split(" / ")[0]}.png', 
                        width=1200, 
                        height=height, 
                        scale=2)
        # fig.show(width=1200, height=height)


In [ ]:
ombudsman = votes.groupby('Qu35').count().rename({1: 'Морозов', 2: 'Варванская'})
ombudsman = ombudsman.loc[['Морозов', 'Варванская']]['ac']


fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=['Данила Морозов', 'Арина Варванская'],
        y=ombudsman.values,
        marker_color=[morozov_color, varvanskaia_color],
        width=0.5,
        text=ombudsman.values,
    )
)
fig.update_layout(
    showlegend=False,
    title_text='<b>Уполномоченный по правам студентов и аспирантов НИУ ВШЭ</b>',
    font_family="Montserrat",
)
fig.write_image(f'results/Уполномоченный по правам студентов и аспирантов НИУ ВШЭ.png', width=800, height=400, scale=3)
    